# Semigroups in GAP

**Website**: http://cloud.gap-system.org

# The Semigroups package
The semigroups package needs to be loaded for the current worksheet.

In [ ]:
LoadPackage("semigroups");

# Transformations

Transformations are to semigroups as permutations are to groups.

Mathematically, permutations are transformations, but in GAP they are not.

In [ ]:
f := Transformation([4, 1, 1, 5, 3, 3]);

In [ ]:
g := Transformation([4, 5, 1, 2, 1, 4]);

In [ ]:
f * g;

In [ ]:
f * g = g * f;

In [ ]:
x := RandomTransformation(10);

In [ ]:
y := RandomTransformation(10);

In [ ]:
z := RandomTransformation(10);

In [ ]:
x * y = y * x; # commutativity

In [ ]:
z * z = z; # idempotent?

Composition of functions is associative...

In [ ]:
(x * y) * z = x * (y * z); # associativity

... and so you can make semigroups out of transformations

In [ ]:
S := Semigroup(x, y, z);

In [ ]:
Size(S);

In [ ]:
t := Transformation([2, 3, 1]);

In [ ]:
p := AsPermutation(t);

In [ ]:
p = t;

In [ ]:
AsPermutation(f);

In [ ]:
t * p;

Permutations and transformations can be multiplied together, but you cannot create a semigroup with both elements.

In [ ]:
Semigroup(t, p);

# IsGroup vs IsMonoid vs IsSemigroup

Something created as a semigroup may not satisfy `IsGroup` even if it is mathematically a group. We use `IsGroupAsSemigroup` to check for this.

There are similar problems with `IsMonoid`. This the analogous property is `IsMonoidAsSemigroup`.

In [ ]:
M := [ [ Z(5), Z(5)^2 ], [ Z(5)^3, Z(5)^2 ] ];

In [ ]:
M in GL(2, 5);

** A group is also a monoid and a semigroup. **

In [ ]:
G := Group(M);

In [ ]:
IsSemigroup(G);

In [ ]:
IsMonoid(G);

In [ ]:
IsGroup(G);

** A semigroup is not (usually) a monoid or a group in GAP -- even if it is mathematically. **

The category depends on how the object was created.

In [ ]:
S := Semigroup(M);

In [ ]:
IsSemigroup(S);

In [ ]:
IsMonoid(S);

In [ ]:
IsMonoidAsSemigroup(S);

In [ ]:
IsGroup(S);

In [ ]:
IsGroupAsSemigroup(S);

In [ ]:
G = S;

For an object in `IsGroup`, GAP needs to know that it can invert any element independently, and it needs how to do it.

For a permutation, there is only ever one inverse, and it is easy to calculate.

A transformation does not necessarily have an inverse, and so GAP needs to consider the whole semigroup to know how to invert an element in a group of transformations. So transformation groups are usually not in the category `IsGroup`, except...

In [ ]:
IsGroup(FullTransformationSemigroup(1));

# Multiplication tables

In [ ]:
A := [
  [1, 2, 3, 4],
  [2, 1, 3, 4],
  [3, 4, 3, 4],
  [4, 3, 3, 4]
];

In [ ]:
S := SemigroupByMultiplicationTable(A);

In [ ]:
T := FullTransformationSemigroup(2);

In [ ]:
IsIsomorphicSemigroup(S, T);

# Creating standard examples

In [ ]:
S := FullTransformationSemigroup(10);

In [ ]:
x := PseudoRandom(S);

In [ ]:
I := SymmetricInverseSemigroup(8);

In [ ]:
x := PseudoRandom(I);

In [ ]:
Print(x);

In [ ]:
P := PartitionMonoid(4);

In [ ]:
x := PseudoRandom(P);

In [ ]:
Print(x);

In [ ]:
M := FullMatrixSemigroup(2, 5);

In [ ]:
x := PseudoRandom(GroupOfUnits(M));

In [ ]:
Print(x);

In [ ]:
AsMatrix(x) in GL(2, 5);

In [ ]:
IsSubsemigroup(M, GL(2, 5));

In [ ]:
x := Random(GL(2, 5));

In [ ]:
NewMatrixOverFiniteField(IsPlistMatrixOverFiniteFieldRep, GF(5), 2, x) in M;

# Finitely presented semigroups

In [ ]:
F := FreeSemigroup("x", "y");

In [ ]:
IsFinite(F);

In [ ]:
x := F.1;

In [ ]:
y := F.2;

**Relation 1**: x \* y = y \* x

In [ ]:
rel1 := [ x * y, y * x ];

**Relation 2**: x ^ 3 = x ^ 2

In [ ]:
rel2 := [ x ^ 3, x ^ 2 ];

**Relation 3**: y ^ 2 = y

In [ ]:
rel3 := [y ^ 2, y ];

In [ ]:
S := F / [rel1, rel2, rel3];

In [ ]:
Size(S);

In [ ]:
Elements(S);

In [ ]:
T := F / [rel2, rel3];

In [ ]:
# IsFinite(T);

# Exhaustive enumeration

In [ ]:
enumerate_semigroup := function(gens)
  local elts, x, g;

  elts := ShallowCopy(gens);
  for x in elts do
    for g in gens do
      if not x * g in elts then
        Add(elts, x * g);
      fi;
    od;
  od;
  return elts;
end;

In [ ]:
Length(enumerate_semigroup([f, g]));

In [ ]:
Size(Semigroup(f, g));

## Create the right Cayley graph during enumeration...

In [ ]:
LoadPackage("digraphs");

In [ ]:
enumerate_with_graph := function(gens)
  local elts, graph, adjacencies, pos, x, g, prod;

  elts := ShallowCopy(gens);
  graph := [];
  for x in elts do
    adjacencies := [];
    for g in gens do
      prod := x * g;
      pos := Position(elts, prod);
      if pos = fail then
        Add(elts, prod);
        Add(adjacencies, Length(elts));
      else
        Add(adjacencies, pos);
      fi;
    od;
    Add(graph, adjacencies);
  od;
  return Digraph(graph);
end;

In [ ]:
gr := enumerate_with_graph([Transformation([1, 1, 3]), Transformation([1, 2, 2])]);

In [ ]:
JUPYTER_DotSplash(DotDigraph(gr));

# Testing commutativity

Commutativity is a nice simple property to check for.

A semigroup is commutative if x * y = y * x for all x, y.

## Version 1
Check all elements for commutativity, as per the definition.

In [ ]:
is_commutative_1 := function(S)
  local x, y;

  for x in S do
    for y in S do
      if not x * y = y * x then
        return false;
      fi;
    od;
  od;
  return true;
end;

In [ ]:
S := FullTransformationSemigroup(4);;

In [ ]:
is_commutative_1(S);

In [ ]:
IsCommutative(S);

In [ ]:
S := Semigroup([
  PartialPerm([1, 2, 3, 4, 5, 6, 8, 9], [1, 2, 3, 4, 5, 6, 8, 9]), 
  PartialPerm([1, 3, 4, 5, 6, 7, 8, 9], [1, 3, 4, 5, 6, 7, 8, 9]), 
  PartialPerm([1, 2, 4, 5, 6, 7, 8, 9], [1, 2, 4, 5, 6, 7, 8, 9]), 
  PartialPerm([1, 2, 3, 4, 6, 7, 8, 9], [1, 2, 3, 4, 6, 7, 8, 9]), 
  PartialPerm([1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8]), 
  PartialPerm([1, 2, 3, 4, 5, 7, 8, 9], [1, 2, 3, 4, 5, 7, 8, 9]), 
  PartialPerm([1, 2, 3, 4, 5, 6, 7, 9], [1, 2, 3, 4, 5, 6, 7, 9]), 
  PartialPerm([2, 3, 4, 5, 6, 7, 8, 9], [2, 3, 4, 5, 6, 7, 8, 9]), 
  PartialPerm([1, 2, 3, 5, 6, 7, 8, 9], [1, 2, 3, 5, 6, 7, 8, 9])]);

In [ ]:
is_commutative_1(S)

In [ ]:
time;

In [ ]:
IsCommutative(S);

In [ ]:
time;

## Version 2
Check all generators

In [ ]:
is_commutative_2 := function(S)
  local gens, x, y;

  gens := GeneratorsOfSemigroup(S);
  for x in gens do
    for y in gens do
      if not x * y = y * x then
        return false;
      fi;
    od;
  od;
  return true;
end;

In [ ]:
is_commutative_2(S);

In [ ]:
time;

### What is the problem with this function?

## Version 3
Don't repeat work!

In [ ]:
is_commutative_3 := function(S)
  local pair;

  for pair in Combinations(GeneratorsOfSemigroup(S), 2) do
    if not pair[1] * pair[2] = pair[2] * pair[1] then
      return false;
    fi;
  od;
  return true;
end;

In [ ]:
is_commutative_3(S);

### Task for the keen: re-implement the above algorithm using `IteratorOfCombinations`.

# Counting idempotents

An idempotent is an element where x ^ 2 = x

## Version 1
Check every element for idempotency.

In [ ]:
nr_idempotents_1 := S -> Number(S, IsIdempotent);

In [ ]:
S := SymmetricInverseSemigroup(7);

In [ ]:
nr_idempotents_1(S);

In [ ]:
time;

In [ ]:
NrIdempotents(S);

In [ ]:
time;

## Version 2
In any semigroup, the number of idempotents is the number of maximal subgroups.

In [ ]:
nr_idempotents_2 := S -> Number(GreensHClasses(S), IsGroupHClass);

In [ ]:
S := SymmetricInverseSemigroup(7);

In [ ]:
nr_idempotents_2(S);

In [ ]:
time;

## Version 3: inverse semigroups
In an inverse semigroup, `NrIdempotents` = `NrRClasses`.

In [ ]:
nr_idempotents_inverse := NrRClasses;

In [ ]:
S := SymmetricInverseSemigroup(7);

In [ ]:
nr_idempotents_inverse(S);

In [ ]:
time;

## Version 4: bands
In a band, every element is idempotent. So `NrIdempotents` = `Size`.

In [ ]:
nr_idempotents_band := Size;

In [ ]:
S := FreeBand(4);

In [ ]:
nr_idempotents_band(S);

In [ ]:
time;

In [ ]:
S := FreeBand(4);

In [ ]:
NrIdempotents(S);

In [ ]:
time;

In [ ]:
InstallMethod(NrIdempotents, "for a band", [IsBand], Size);

In [ ]:
S := FreeBand(4);

In [ ]:
SetIsBand(S, true);

In [ ]:
NrIdempotents(S);

In [ ]:
time;